In [1]:
!pip install transformers

import pandas as pd
import os
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 84.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 16.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/CS\ 224N/CS\ 224N\ Project
! ls # verify that you are in the right directory

Mounted at /content/drive
/content/drive/MyDrive/CS 224N/CS 224N Project
'224N Project Brainstorm.gdoc'	        dataset_agg.ipynb
'224N Project Helpful Tutorials.gdoc'  'first proposal OLD'
'224N Project Milestone Notes.gdoc'     gpt2-baseline.ipynb
 aita_clean.csv			        logs
 aita_comments.csv		       'Reddit Scraper.ipynb'
 aita_test_set.csv		        results
 aita_train_set.csv		        T5-baseline.ipynb
 aita_valid_set.csv		        t5_checkpoints
 bart-baseline-attempt2.ipynb	        train.pt
 bert-baseline.ipynb		        wandb
 csvs


In [3]:
train_data = pd.read_csv('aita_train_set.csv')
valid_data = pd.read_csv('aita_valid_set.csv')
test_data = pd.read_csv('aita_test_set.csv')

In [59]:
# class RedditText(Dataset):  
#     def __init__(self,control_code, truncate=False, gpt2_type="gpt2", max_length=1024):

#         self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
#         self.texts = []
#         self.comments = []

#         for row in valid_data['text']:
#           self.comments.append(torch.tensor(
#                 self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
#             ))               

#         # for index, row in valid_data.iterrows():
#           #  self.texts.append(torch.tensor(
#           #       self.tokenizer.encode(f"<|start|>{row['text'][:max_length]}<|endoftext|>")
#           #   ))  
#           #  self.comments.append(torch.tensor(
#           #       self.tokenizer.encode(f"<|start|>{row['comments'][:max_length]}<|endoftext|>")
#           #   ))  
#           # self.texts.append(torch.tensor(
#           #       self.tokenizer.encode(f"<|{control_code}|>{row['text']}<|endoftext|>"[:max_length])
#           #   ))  
#           # self.comments.append(torch.tensor(
#           #       self.tokenizer.encode(f"<|{control_code}|>{row['comments'][:max_length]}<|endoftext|>")
#           #   ))  
          
#         if truncate:
#             self.texts = self.texts[:20000]
#         self.texts_count = len(self.texts)
        
#     def __len__(self):
#         return self.texts_count

#     def __getitem__(self, item):
#         comment = self.comments[item]
#         text = self.texts[item]
#         return text
    
# dataset = RedditText(valid_data["text"], truncate=False, gpt2_type="gpt2")  

In [108]:

class RedditText(Dataset):  
    def __init__(self, control_code, truncate=False, gpt2_type="gpt2", max_length=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.texts = []
        self.comments = []
        self.comments_len = []

        for row in valid_data['text']:
          s = f"<start>{row[:max_length - 20]}<|endoftext|>"
          s += '.' * int(max_length - len(s))
          self.texts.append(torch.tensor(
                self.tokenizer.encode(s)
            ))   
          # print(self.texts[-1].size())
          # break
        for row in valid_data['comments']:
          
          s = f"<|start|>{row[:max_length]}<|endofcomment|>"
          s += '.' * int(max_length - len(s))
          self.comments.append(torch.tensor(
                self.tokenizer.encode(s)
            ))   
          # print(self.comments[-1].size())
        if truncate:
            self.texts = self.texts[:20000]
        self.texts_count = len(self.texts)
        
    def __len__(self):
        return self.texts_coun

    def __getitem__(self, item):
        return self.texts[item], self.comments[item]
    
dataset = RedditText(valid_data['text'], truncate=False, gpt2_type="gpt2")      

In [13]:
# # torch.save(dataset, './train.pt')
# # from train import RedditText
# data = RedditText()
# data = torch.load('./train.pt') 
# print(data)

In [104]:
#Get the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [46]:
# def train(
#     dataset, model, tokenizer,
#     batch_size=16, epochs=5, lr=2e-5,
#     max_seq_len=400, warmup_steps=200,
#     gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
#     test_mode=False,save_model_on_epoch=False,
# ):
#     acc_steps = 100
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     # if on gpu uncomment the below two lines
#     model = model.to(device)
#     model.train()

#     optimizer = AdamW(model.parameters(), lr=lr)
#     scheduler = get_linear_schedule_with_warmup(
#         optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
#     )

#     train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
#     loss=0
#     accumulating_batch_count = 0
#     input_tensor = None

#     for epoch in range(epochs):

#         print(f"Training epoch {epoch}")
#         print(loss)
#         for idx, entry in tqdm(enumerate(train_dataloader)):
#             (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

#             if carry_on and idx != len(train_dataloader) - 1:
#                 continue

#             # if on cuda gpu uncomment the below line
#             input_tensor = input_tensor.to(device)
#             # print(input_tensor)
#             # print(outputs)
#             outputs = model(input_tensor, labels=entry)
#             loss = outputs[0]
#             loss.backward()
#             if idx % 100 == 0:
#               print(f'index number: {idx} and loss: {loss.item()}')

#             if (accumulating_batch_count % batch_size) == 0:
#                 optimizer.step()
#                 scheduler.step()
#                 optimizer.zero_grad()
#                 model.zero_grad()

#             accumulating_batch_count += 1
#             input_tensor = None
#         if save_model_on_epoch:
#             torch.save(
#                 model.state_dict(),
#                 os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
#             )
#     return model

In [109]:
def train(
    dataset, model, tokenizer,
    batch_size=16, epochs=5, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):
    acc_steps = 100
    # device=torch.device("cuda")
    # model = model.cuda()
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry[0], input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            # input_tensor = input_tensor.to(device)
            print(input_tensor.size())
            print(entry[1].size())
            outputs = model(input_tensor, labels=entry[1])
            loss = outputs[0]
            loss.backward()
            if idx % 10 ==0:
              print(loss.item())

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [1]:
model = train(dataset, model, tokenizer, epochs=1, save_model_on_epoch=True)


NameError: ignored

In [ ]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=50, #maximum number of words to generate based on avg comment length
    top_p=0.8,
    temperature=1.,
):
    model.eval()
    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
              generated_list.append(output_text)
                
    return generated_list

#Function to generate multiple sentences. Test data should be a dataframe
def text_generation(test_data):
  generated_texts = []
  for i in range(len(test_data)):
    x = generate(model.to('cpu'), tokenizer, test_data['text'][i], entry_count=1)
    generated_texts.append(x)
  return generated_texts

#Run the functions to generate the texts
generated_texts = text_generation(test_data)

In [ ]:
my_generations=[]

# for i in range(len(generated_texts)):
#   # a = test_data['Lyric'][i].split()[-30:] #Get the matching string we want (30 words)
#   a = test_data['comments'][i] #Get the matching string we want (30 words)

  # b = ' '.join(a)
  # c = ' '.join(generated_texts[i]) #Get all that comes after the matching string
  # my_generations.append(c.split(b)[-1])
my_generations = generated_texts
test_data['generated_comments'] = my_generations


#Finish the sentences when there is a point, remove after that
final=[]

for i in range(len(test_data)):
  to_remove = test_data['generated_comments'][i].split('.')[-1]
  final.append(test_data['generated_comments'][i].replace(to_remove,''))

test_data['generated_comments'] = final

In [ ]:
import statistics
from nltk.translate.bleu_score import sentence_bleu

scores=[]

for i in range(len(test_data)):
  reference = test_data['comments'][i]
  candidate = test_data['generated_comments'][i]
  scores.append(sentence_bleu(reference, candidate))

statistics.mean(scores)